# Quantum Teleportation:
Without going into much detail, let's discuss only a tricky part of this topic (where I stambled to be more specific!)

Below are 4 Bell states which are maximally entangled and build on a orthogonal basis:
- <ψ<sup>00</sup>> = $\frac{1}{\sqrt{2}}$ * (|00> + |11>)     
- <ψ<sup>10</sup>> = $\frac{1}{\sqrt{2}}$ * (|00> - |01>)  
- <ψ<sup>01</sup>> = $\frac{1}{\sqrt{2}}$ * (|01> + |10>)  
- <ψ<sup>11</sup>> = $\frac{1}{\sqrt{2}}$ * (|01> - |10>)  


So, the general state can be written as:  <ψ<sup>ij</sup>> = (I<sub>2</sub> $\bigotimes$ $\sigma$<sub>x</sub><sup>j</sup>> $\sigma$<sub>z</sub><sup>i</sup>>) <ψ<sup>00</sup>> 

## Discussion

Alice and Bob both shares a maximally entangled state. i.e: some <ψ<sup>ij</sup>> where i == j == 0 for our example ==> <ψ<sup>00</sup>><sub>AB</sub>

Consider a qubit S that Alice possesses, which is in the following state:

|$\phi$⟩<sub>s</sub> = α|0⟩<sub>s</sub> +β|1⟩<sub>s</sub>

The combined state of SAB:
|$\phi$⟩ $\bigotimes$  <ψ<sup>00</sup>><sub>AB</sub> ==> <ψ<sup>ij</sup>><sub>AS</sub>&ensp;$\sigma$<sub>x</sub><sup>j</sup> $\sigma$<sub>z</sub><sup>i</sup>&ensp;|$\phi$⟩<sub>B</sub>
 
Alice performs a Bell measurement on her qubit of a shared state and the state she want to send to Bob. After measurment Alice's state AS will collapse to the one of the following states. Note: (this can be shown by re arranging the combined state of SAB, spaces are added for a better look and understanding): 
- <ψ<sup>00</sup>><sub>AS</sub>&ensp;$\sigma$<sub>x</sub><sup>0</sup> $\sigma$<sub>z</sub><sup>0</sup>&ensp;|$\phi$⟩<sub>B</sub>
- <ψ<sup>01</sup>><sub>AS</sub>&ensp;$\sigma$<sub>x</sub><sup>1</sup> $\sigma$<sub>z</sub><sup>0</sup>&ensp;|$\phi$⟩<sub>B</sub>
- <ψ<sup>10</sup>><sub>AS</sub>&ensp;$\sigma$<sub>x</sub><sup>0</sup> $\sigma$<sub>z</sub><sup>1</sup>&ensp;|$\phi$⟩<sub>B</sub>
- <ψ<sup>11</sup>><sub>AS</sub>&ensp;$\sigma$<sub>x</sub><sup>1</sup> $\sigma$<sub>z</sub><sup>1</sup>&ensp;|$\phi$⟩<sub>B</sub>

## Tricky part:
- Bell measurement on Alice side gets performed on state A and S only
- So, XOR with Hadamard on Alice's qubits will give her classical bits i and j (after Bell measurement on SA)
- Apparently Bob's qubit's state can be describe using a general version ==>  $\sigma$<sub>x</sub><sup>j</sup> $\sigma$<sub>z</sub><sup>i</sup>&ensp;|$\phi$⟩<sub>B</sub> (due to combined state of SAB)
- Now, we have i and j, and Bob's state
- After Bell measurement, Alice's state will get collapsed to one of the 4 possible states and Bob will get i and j values (Alice's both qubit gets destroyed, of course!)
- After Bell measurement, Alice will know the exact values of i and j and set them to Bob
- So, now Bob has i and j values (sent by Alice). In order to convert Bob's state to |$\phi$⟩<sub>B</sub>, Bob requires to perform some calculation
- Those calculations depends on the values of i and j (sent by Alice). Let's see Bob's state ==> $\sigma$<sub>x</sub><sup>j</sup> $\sigma$<sub>z</sub><sup>i</sup>&ensp;|$\phi$⟩<sub>B</sub>, 
- Bob will use i and j given by Alcie to convert his state to |$\phi$⟩<sub>B</sub> 
- Why? Well, if we apply same gate twice, we will get the original state back i.e: |$\phi$⟩<sub>B</sub>